# Collect demonstrations

Interactive notebook for collecting demonstrations

In [ ]:
# %matplotlib auto

import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import socket
from datetime import datetime
from demonstration_recorder import DemonstrationRecorder

from robot.al5d_position_controller import PositionController
from robot.al5d_simulated_position_controller import SimulatedPositionController


from camera.camera_controller import CameraController
from demonstration import Demonstration


In [ ]:
experiment = "demonstration_collector"
# picking up a collection specific name
hostname = socket.gethostname()
print(f"Hostname is {hostname}")
if hostname == "raven":
    run = "collect_freeform_raven"
elif hostname == "szenes.local":
    run = "collect_no_robot"
elif hostname == "tredy2":
    run = "tredy2_collect_freeform"
    #run = "tredy2_automove_rwp6d_00"
else:
    run = "collect_freeform"

exp = Config().get_experiment(experiment, run)
exp_target = Config().get_experiment("demonstration", exp["demonstration_name"])
exp_position_controller = Config().get_experiment(
    exp["exp_position_controller"], 
    exp["run_position_controller"])
exp_camera_controller = Config().get_experiment(
    exp["exp_camera_controller"], exp["run_camera_controller"])



In [ ]:
# This was here to debug things, because it was not finding one of the cameras
import cv2

# identifying which cameras work
for i in range(10):
    cap = cv2.VideoCapture(i)
    # FIXME: set for the specified one
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 256)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 256)
    cap.set(cv2.CAP_PROP_FPS, 10)

    ret, frame = cap.read()
    if ret:
        print(f"Camera {i} is working")
    else:
        print(f"Camera {i} not working")
    cap.release()

In [ ]:
# starting the robot controller
if exp["available_robot"]:
    robot_controller = PositionController(exp_position_controller)
else:
    robot_controller = SimulatedPositionController(exp_position_controller)
# starting the camere controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True


task_dir = exp_target.data_dir()
demoname = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')
demo_dir = pathlib.Path(task_dir, demoname)
demo_dir.mkdir(parents=False, exist_ok=False)
print(f"Ok, demonstration with go into\n\t {demo_dir}")

demonstration = Demonstration(exp_target, demoname, parse_old_style = False)
demonstration.metadata["cameras"] = sorted(camera_controller.capture_devs.keys())
demonstration.save_metadata()
# fixme need to add the cameras into the demonstration

In [ ]:
# Performs the recording. The robot is controlled through the primary 
# controller. This is also used to terminate the collection
# The information is collected by the demonstration recorder

controller = exp["primary_controller"]
    

if controller == "xbox":
    # the gamepad is only available on some machines, and only on Linux
    available_gamepad = True
    try:
        from remote_control.gamepad_controller import GamepadController
    except ModuleNotFoundError:
        print("Approxeng module not found, cannot use gamepad")
        available_gamepad = False
    exp_gamepad_controller = Config().get_experiment(exp["exp_gamepad_controller"], exp["run_gamepad_controller"])
    gamepad_controller = GamepadController(exp_gamepad_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration,
        camera_controller=camera_controller, remote_control=gamepad_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    gamepad_controller.demonstration_recorder = demo_recorder
    gamepad_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")
if controller == "keyboard":
    exp_keyboard_controller = Config().get_experiment(exp["exp_keyboard_controller"], exp["run_keyboard_controller"])
    from remote_control.keyboard_controller import KeyboardController
    kb_controller = KeyboardController(exp_keyboard_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=kb_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    kb_controller.demonstration_recorder = demo_recorder
    kb_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")

if controller == "widowx":
    from remote_control.widowx_controller import WidowXController
    exp_widowx_controller = Config().get_experiment(exp["exp_widowx_controller"], exp["run_widowx_controller"])    
    widowx_controller = WidowXController(exp_widowx_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=widowx_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    widowx_controller.demonstration_recorder = demo_recorder
    widowx_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")

if controller == "camera":
    # FIXME: future controller which is based on a camera and capturing human movement etc.
    pass

if controller == "automove":
    from remote_control.automove_controller import AutoMoveController
    exp_automove_controller = Config().get_experiment(exp["exp_automove_controller"], exp["run_automove_controller"])
    automove_controller = AutoMoveController(exp_automove_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    automove_controller.generate_waypoints()
    # program_controller.interactive_confirm = False
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=automove_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    automove_controller.demonstration_recorder = demo_recorder
    demonstration.save_metadata()
    automove_controller.control()



In [ ]:
demonstration.save_metadata()
demonstration.move_to_video(delete_img_files=True)


In [ ]:
print(demonstration.metadata)

In [ ]:
exp